In [1]:
import pandas as pd
import numpy as np
from utils import sigmoid, predict, sigmoid_cross_entropy_with_logits, sigmoid_cross_entropy_with_x_w, sigmoid_cross_entropy_truncated, derivative_cost_wrt_params, backtracking_line_search

In [2]:
from sklearn.model_selection import StratifiedKFold

In [3]:
csv_path = r"C:\Users\Thanh\Downloads\voice_gender\voice.csv"

In [4]:
df = pd.read_csv(csv_path)

In [5]:
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [6]:
df['label']=df['label'].replace({'male':1,'female':0})

In [7]:
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,1
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,1
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,1


In [8]:
x = df.drop("label", axis=1).to_numpy(dtype=np.float)
y = df["label"].values.astype(np.float)

In [9]:
x = (x - np.min(x, axis=0, keepdims=True))/(np.max(x, axis=0, keepdims=True) - np.min(x, axis=0, keepdims=True))
#x = (x-np.mean(x, axis=0, keepdims=True))/np.std(x, axis=0, keepdims=True)

In [10]:
x

array([[9.64185977e-02, 4.73408557e-01, 8.41252523e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.25828039e-01, 5.05074648e-01, 1.16899775e-01, ...,
        2.14438885e-03, 2.14592275e-03, 5.64489929e-02],
       [1.79222164e-01, 6.75535813e-01, 1.02872692e-01, ...,
        3.57398142e-04, 3.57653791e-04, 4.98851565e-02],
       ...,
       [4.84948580e-01, 7.99042208e-01, 6.90336544e-01, ...,
        1.34024303e-01, 1.34120172e-01, 2.08884632e-01],
       [4.92516043e-01, 7.45692037e-01, 6.95311295e-01, ...,
        1.64045747e-01, 1.64163090e-01, 3.33559435e-01],
       [5.95699639e-01, 7.68963896e-01, 6.87590032e-01, ...,
        2.50178699e-02, 2.50357654e-02, 3.75385802e-01]])

In [11]:
x.shape

(3168, 20)

In [12]:
ones = np.ones(shape=[x.shape[0], 1], dtype=np.float)
x = np.append(x, ones, axis=1)
x.shape

(3168, 21)

In [13]:
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(x, y)

5

In [14]:
for train_index, test_index in skf.split(x, y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [15]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2535, 21), (633, 21), (2535,), (633,))

In [16]:
#weights = np.random.normal(loc=0, scale=np.sqrt(2/(x.shape[1] + 1)), size=x.shape[1])
weights = np.random.normal(loc=0, scale=np.sqrt(2/(x.shape[1])), size=x.shape[1]-1)
weights = np.append(weights, 0.)

In [17]:
weights

array([ 0.05192809, -0.2479905 , -0.6676255 , -0.00627699, -0.62320055,
        0.7993867 ,  0.11597334, -0.12272189,  0.01868447,  0.47526641,
        0.31790324, -0.03016008,  0.32616962, -0.10767109, -0.22006825,
        0.36979197,  0.38905149, -0.21934269,  0.14828924, -0.0149664 ,
        0.        ])

In [18]:
learning_rate = 5

In [19]:
derivative_cost_wrt_params(x=x, w=weights, y=y).shape

(21,)

In [20]:
sigmoid_cross_entropy_with_x_w(x=x, w=weights, y=y)

0.684189217694314

In [21]:
x = x_train
y = y_train
min_val_cost = np.inf
patience = 0
max_weights = weights
for i in range(10000):
    dweights = derivative_cost_wrt_params(x=x, w=weights, y=y)
    #print("dweights: {}".format(dweights))
    #dweights = np.linalg.norm(dweights)
    #if i % 1000 == 0:
    #    learning_rate *= 0.9
    #weights -= learning_rate * dweights
    alpha = backtracking_line_search(x=x, w=weights, y=y, p=-dweights, rho=0.9, alpha=10, c=0.0001)
    weights -= alpha * dweights
    #print("Alpha: {}".format(alpha))
    #print(weights)
    cost = sigmoid_cross_entropy_with_x_w(x=x, w=weights, y=y)
    val_cost = sigmoid_cross_entropy_with_x_w(x=x_test, w=weights, y=y_test)
    #output = predict(x=x, w=weights, threshold=0.6).astype(np.float)
    #acc = (output == y).sum()/y.shape[0]
    tr_output = predict(x=x, w=weights, threshold=0.6).astype(np.float)
    tr_acc = (tr_output == y).sum()/y.shape[0]
    val_output = predict(x=x_test, w=weights, threshold=0.6).astype(np.float)
    val_acc = (val_output == y_test).sum()/y_test.shape[0]
    #print(output == y)
    #print(cost, acc)
    print(cost, val_cost, tr_acc, val_acc, patience)
    if val_cost < min_val_cost:
        min_val_cost = val_cost
        patience = 0
        max_weights = weights
    else:
        patience += 1

    #if patience >= 20 and np.allclose(dweights, np.zeros_like(dweights), rtol=1e-3, atol=1e-5):
    if patience >= 20:
        weights = max_weights
        print("Stop at epoch {}, restore weights".format(i))
        break


tr_output = predict(x=x, w=weights, threshold=0.6).astype(np.float)
tr_acc = (tr_output == y).sum()/y.shape[0]
val_output = predict(x=x_test, w=weights, threshold=0.6).astype(np.float)
val_acc = (val_output == y_test).sum()/y_test.shape[0]
print(cost, val_cost, tr_acc, val_acc)
print(dweights, np.linalg.norm(dweights))

0.676440180468568 0.7094677063348958 0.7751479289940828 0.7109004739336493 0
0.6650720151296463 0.715211687656517 0.5001972386587771 0.5007898894154819 0
0.6500223356894578 0.69856290817401 0.8047337278106509 0.7187993680884676 1
0.6389143505687852 0.7003602310940346 0.5025641025641026 0.4976303317535545 0
0.620510330446209 0.680941762183626 0.834319526627219 0.7361769352290679 1
0.6070120851141785 0.6761423557435015 0.5214990138067062 0.4960505529225908 0
0.5870964678447481 0.6561608996062832 0.8544378698224852 0.7503949447077409 0
0.5725169761833613 0.6466722145223434 0.5435897435897435 0.4928909952606635 0
0.5702576978664511 0.6458450087152711 0.8583826429980276 0.7551342812006319 0
0.5522988384055003 0.630680703878279 0.5652859960552268 0.5023696682464455 0
0.5468927012491447 0.6269253939300986 0.8635108481262327 0.7740916271721959 0
0.5436019382428503 0.625641237006699 0.5783037475345167 0.5071090047393365 0
0.5341762264399856 0.6178464352176861 0.8646942800788955 0.78199052132701

In [22]:
weights

array([  1.38443971,   1.52150222,  -1.82773335,  -2.12687362,
         3.37678943,   7.32914082,  -1.34401808,  -1.28352287,
         5.19698755,  -4.10579664,   0.55866559,   1.30235154,
       -22.59737087,   2.27349138,  -0.41708503,   0.42594102,
         0.78054566,   0.04129543,   0.401479  ,  -0.57083097,
         3.75649224])